In [47]:
import sys
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install chart_studio
!{sys.executable} -m pip install seaborn

import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import random

import plotly.express as px
import plotly.graph_objs as go
import chart_studio.plotly as py
import chart_studio.tools as tls
import os

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
%env PLOTLY_USER=arhanna
%env PLOTLY_API=6EStzo8mO5ZY6kfaehPe

env: PLOTLY_USER=arhanna
env: PLOTLY_API=6EStzo8mO5ZY6kfaehPe


In [6]:
user = %env PLOTLY_USER
api = %env PLOTLY_API

tls.set_credentials_file(username= user, api_key= api)

# Data Loading Stage + Pre-processing

In [7]:
# Custom dataset
# Added custom column header line to the csv after download
df = pd.read_csv('./data.csv')
df['created_at'] = pd.to_datetime(df['created_at'])
df.describe()

,id,confidence
count,4.990000e+02,499.000000
mean,1.361428e+18,0.714783
std,5.215730e+15,0.128498
min,1.349833e+18,0.500376
25%,1.358297e+18,0.601505
50%,1.363227e+18,0.707502
75%,1.366536e+18,0.823958
max,1.366980e+18,0.997732


In [8]:
df.head()

,text,user,id,created_at,confidence,sentiment
0,RT @Tesla: Model 3 earns Top Safety+ award fro...,elonmusk,1366451032356610050,2021-03-01 18:11:28+00:00,0.652028,Positive
1,@thesheetztweetz Ahem …,elonmusk,1366446908684705792,2021-03-01 17:55:05+00:00,0.839178,Positive
2,https://t.co/jJeP4pFR6m,elonmusk,1365241433381339136,2021-02-26 10:04:57+00:00,0.589783,Positive
3,Do androids dream of electric cars?,elonmusk,1365237858165026818,2021-02-26 09:50:44+00:00,0.504706,Positive
4,@Sharvenium @flcnhvy 🤣🤣,elonmusk,1365226460244561920,2021-02-26 09:05:27+00:00,0.589783,Positive


### Setup groups/counts/sums

In [24]:
# df_s: change sentiment to ints: 1,-1
df_s = df.copy()
# df_s['sentiment'] = df_s['sentiment'].apply(lambda x: 1 if x=="Positive" else -1)
df_s.head()

,text,user,id,created_at,confidence,sentiment
0,RT @Tesla: Model 3 earns Top Safety+ award fro...,elonmusk,1366451032356610050,2021-03-01 18:11:28+00:00,0.652028,Positive
1,@thesheetztweetz Ahem …,elonmusk,1366446908684705792,2021-03-01 17:55:05+00:00,0.839178,Positive
2,https://t.co/jJeP4pFR6m,elonmusk,1365241433381339136,2021-02-26 10:04:57+00:00,0.589783,Positive
3,Do androids dream of electric cars?,elonmusk,1365237858165026818,2021-02-26 09:50:44+00:00,0.504706,Positive
4,@Sharvenium @flcnhvy 🤣🤣,elonmusk,1365226460244561920,2021-02-26 09:05:27+00:00,0.589783,Positive


In [25]:
df_g = df_s.copy()
df_g_sentiment = df_s.groupby(['user', pd.Grouper(key='created_at', freq='D')])['sentiment'].sum().reset_index(name ='sentiment')
df_g.head()
# df_g.describe()

,text,user,id,created_at,confidence,sentiment
0,RT @Tesla: Model 3 earns Top Safety+ award fro...,elonmusk,1366451032356610050,2021-03-01 18:11:28+00:00,0.652028,Positive
1,@thesheetztweetz Ahem …,elonmusk,1366446908684705792,2021-03-01 17:55:05+00:00,0.839178,Positive
2,https://t.co/jJeP4pFR6m,elonmusk,1365241433381339136,2021-02-26 10:04:57+00:00,0.589783,Positive
3,Do androids dream of electric cars?,elonmusk,1365237858165026818,2021-02-26 09:50:44+00:00,0.504706,Positive
4,@Sharvenium @flcnhvy 🤣🤣,elonmusk,1365226460244561920,2021-02-26 09:05:27+00:00,0.589783,Positive


In [26]:
df_g_sentiment.head()

,user,created_at,sentiment
0,benshapiro,2021-03-02 00:00:00+00:00,PositivePositiveNegativePositiveNegativePositi...
1,benshapiro,2021-03-03 00:00:00+00:00,NegativePositiveNegativePositivePositivePositi...
2,elonmusk,2021-02-06 00:00:00+00:00,NegativePositivePositivePositivePositivePositi...
3,elonmusk,2021-02-07 00:00:00+00:00,PositiveNegativeNegativePositivePositivePositi...
4,elonmusk,2021-02-08 00:00:00+00:00,NegativePositivePositivePositivePositiveNegati...


In [41]:
# fig, ax = plt.subplots(figsize=(30,10))
cdm = {"Positive": "Blue", "Negative": "Red"}
co ={"sentiment": ["Positive", "Negative"]}
df_user_sum = df.groupby(['user', 'sentiment'])['sentiment'].count().reset_index(name='count')
fig = px.bar(df_user_sum, x="user", y="count",color="sentiment", title="User Total", category_orders=co)
url = py.plot(fig, filename='user_tweets',auto_open=False)
df_user_sum.head()

,user,sentiment,count
0,benshapiro,Negative,36
1,benshapiro,Positive,64
2,elonmusk,Negative,48
3,elonmusk,Positive,152
4,sza,Negative,78


### Specific User analysis

In [94]:
user = random.choice(df.user.unique())
df_u = df_s[df_s['user']==user].groupby(['user', 'sentiment',pd.Grouper(key='created_at', freq='D')])['sentiment'].count().reset_index(name='total')
df_u['nroll_7'] = df_u[df_u['sentiment']=='Negative'].total.rolling(7,win_type='triang').mean()
df_u['proll_7'] = df_u[df_u['sentiment']=='Positive'].total.rolling(7,win_type='triang').mean()

df_u['nroll_7'] = df_u['nroll_7'].fillna(0)
df_u['proll_7'] = df_u['proll_7'].fillna(0)

# df_u['sentiment'] = df_u['sum'].apply(lambda x: "Negative" if x < 0 else "Positive")
df_u.head(20)
df_u[df_u['sentiment']=='Positive'].head(30)

,user,sentiment,created_at,total,nroll_7,proll_7
31,sza,Positive,2021-01-14 00:00:00+00:00,2,0.0,0.0000
32,sza,Positive,2021-01-16 00:00:00+00:00,8,0.0,0.0000
33,sza,Positive,2021-01-18 00:00:00+00:00,3,0.0,0.0000
34,sza,Positive,2021-01-19 00:00:00+00:00,2,0.0,0.0000
35,sza,Positive,2021-01-20 00:00:00+00:00,1,0.0,0.0000
36,sza,Positive,2021-01-21 00:00:00+00:00,8,0.0,0.0000
37,sza,Positive,2021-01-22 00:00:00+00:00,1,0.0,3.4375
38,sza,Positive,2021-01-23 00:00:00+00:00,11,0.0,3.8125
39,sza,Positive,2021-01-24 00:00:00+00:00,10,0.0,4.8125
40,sza,Positive,2021-01-25 00:00:00+00:00,3,0.0,5.5000


In [95]:
fig = px.bar(df_u, x="created_at", y="total", color="sentiment", barmode="group",category_orders=co, title=f'{user}: Tweets',hover_data=df_u.columns)
fig.add_trace(go.Scatter(x=df_u[df_u['sentiment']=='Negative']['created_at'],y=df_u[df_u['sentiment']=='Negative']['nroll_7'], name='nroll_7'))
fig.add_trace(go.Scatter(x=df_u[df_u['sentiment']=='Positive']['created_at'],y=df_u[df_u['sentiment']=='Positive']['proll_7'], name='proll_7'))

url = py.plot(fig, filename=f'{user}_bar')

In [15]:
print(url)

https://plotly.com/~arhanna/54/


#### Time Series With two bars per day of negative/positive sum

In [52]:
df_u.total.rolling(7,win_type='triang').mean()

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6     3.3125
7     3.3125
8     2.8750
9     2.4375
10    2.0625
11    1.8125
12    1.8125
13    1.8125
14    1.6875
15    1.6250
16    1.5625
17    1.6250
18    1.8125
19    2.0625
20    2.3750
21    2.3750
22    2.5625
23    3.1250
24    3.8750
25    5.0625
26    6.1250
27    6.5000
28    6.3750
29    5.8750
30    4.8125
31    3.9375
32    3.5625
33    3.1875
34    3.2500
35    3.6250
36    3.8750
37    4.5625
38    5.6250
39    6.2500
40    6.8125
41    7.6250
42    7.9375
43    8.6875
44    9.3125
45    9.0000
Name: total, dtype: float64

In [88]:
words = [('this', [('that', 0.5935733914375305), ('which', 0.5301090478897095), ('the', 0.49837833642959595), ('it', 0.47747719287872314), ('every', 0.4759232997894287), ('next', 0.4756498634815216), ('another', 0.40533578395843506), ('a', 0.40285879373550415), ('today', 0.3960303068161011), ('ths', 0.39152267575263977)]), ('is', [("isn't", 0.7290782928466797), ('isnt', 0.6854735612869263), ('keeps', 0.6094664335250854), ('was', 0.5938674211502075), ('gets', 0.5444707870483398), ("she's", 0.5371303558349609), ("wasn't", 0.5240029692649841), ("he's", 0.5117640495300293), ('shes', 0.5107032060623169), ('hes', 0.5040029287338257)]), ('a', [('another', 0.6717653274536133), ('this', 0.40285879373550415), ('the', 0.3975605070590973), ('every', 0.3918810486793518), ('parking', 0.37662768363952637), ('huge', 0.37332087755203247), ('that', 0.349668025970459), ('massive', 0.33672189712524414), ("yesterday's", 0.3366018533706665), ('something', 0.33431485295295715)]), ('test', [('exam', 0.8069786429405212), ('maths', 0.7357074022293091), ('assignment', 0.7338716387748718), ('math', 0.7284610271453857), ('essay', 0.7228487133979797), ('chem', 0.7096487879753113), ('lesson', 0.7095023393630981), ('paper', 0.7084890007972717), ('physics', 0.7060012817382812), ('biology', 0.7050876617431641)]), ('of', [('current', 0.4695361852645874), ("world's", 0.435147225856781), ('valuable', 0.4214920103549957), ('stanley', 0.42116615176200867), ('involving', 0.41132527589797974), ('recent', 0.4069414436817169), ('gun', 0.3961132764816284), ('led', 0.3896802067756653), ('ov', 0.3890407085418701), ('tree', 0.3803018033504486)]), ('the', [('our', 0.5865063667297363), ('their', 0.5239236354827881), ('teh', 0.5046848058700562), ('this', 0.49837833642959595), ('security', 0.466427743434906), ('my', 0.46495282649993896), ('india', 0.46466588973999023), ('that', 0.4457013010978699), ('liverpool', 0.4425622224807739), ('main', 0.4424588680267334)]), ('word', [('question', 0.6391710042953491), ('phrase', 0.6108942031860352), ('plot', 0.6053066253662109), ('sentence', 0.6003183126449585), ('subject', 0.6000585556030273), ('language', 0.5994502902030945), ('character', 0.59331876039505), ('code', 0.5902299284934998), ('situation', 0.5838054418563843), ('words', 0.580930233001709)]), ('for', [('4', 0.5169053077697754), ('patiently', 0.4448145627975464), ('upon', 0.4136114716529846), ('after', 0.3829094469547272), ('during', 0.3789081871509552), ('anxiously', 0.36868807673454285), ('extra', 0.3670605421066284), ('forward', 0.36466094851493835), ('bible', 0.36188894510269165), ('bills', 0.3543490767478943)]), ('negativity', [('raul', 0.8106224536895752), ('jumpy', 0.8050857782363892), ('filipinos', 0.7945456504821777), ('brando', 0.7900981903076172), ('nn', 0.7812931537628174), ('m-f', 0.7798870801925659), ('chicky', 0.7786859273910522), ('kenneth', 0.7747387290000916), ('baaaaaack', 0.7745330333709717), ('jajajaja', 0.774107813835144)]), ('and', [('&', 0.6769999265670776), ('but', 0.6172769665718079), ('because', 0.5828579664230347), ('.', 0.560845136642456), ('when', 0.5040073394775391), ('cause', 0.48166900873184204), (',', 0.4780077338218689), ('cos', 0.47045964002609253), ('whilst', 0.4475087523460388), ('cuz', 0.4334430992603302)]), ('stuff', [('things', 0.6873272657394409), ('clothes', 0.6454428434371948), ('stuffs', 0.6232566833496094), ('bits', 0.5912203788757324), ('information', 0.570555567741394), ('photos', 0.567740261554718), ('books', 0.5676676034927368), ('projects', 0.5570785999298096), ('furniture', 0.547055184841156), ('pictures', 0.5450241565704346)]), ('.', [(',', 0.7772355079650879), ('...', 0.7494542598724365), ('because', 0.5960568189620972), ('but', 0.5703451037406921), ('and', 0.560845136642456), ('cause', 0.5266448259353638), ('..', 0.504142701625824), ('cos', 0.4608926475048065), ('that', 0.44617965817451477), ('cuz', 0.443237841129303)])]

In [89]:
print(words)

[('this', [('that', 0.5935733914375305), ('which', 0.5301090478897095), ('the', 0.49837833642959595), ('it', 0.47747719287872314), ('every', 0.4759232997894287), ('next', 0.4756498634815216), ('another', 0.40533578395843506), ('a', 0.40285879373550415), ('today', 0.3960303068161011), ('ths', 0.39152267575263977)]), ('is', [("isn't", 0.7290782928466797), ('isnt', 0.6854735612869263), ('keeps', 0.6094664335250854), ('was', 0.5938674211502075), ('gets', 0.5444707870483398), ("she's", 0.5371303558349609), ("wasn't", 0.5240029692649841), ("he's", 0.5117640495300293), ('shes', 0.5107032060623169), ('hes', 0.5040029287338257)]), ('a', [('another', 0.6717653274536133), ('this', 0.40285879373550415), ('the', 0.3975605070590973), ('every', 0.3918810486793518), ('parking', 0.37662768363952637), ('huge', 0.37332087755203247), ('that', 0.349668025970459), ('massive', 0.33672189712524414), ("yesterday's", 0.3366018533706665), ('something', 0.33431485295295715)]), ('test', [('exam', 0.806978642940521

In [90]:
words = list(words)

In [91]:
print(len(words))

12


In [92]:
for x in words:
    print(x)

('this', [('that', 0.5935733914375305), ('which', 0.5301090478897095), ('the', 0.49837833642959595), ('it', 0.47747719287872314), ('every', 0.4759232997894287), ('next', 0.4756498634815216), ('another', 0.40533578395843506), ('a', 0.40285879373550415), ('today', 0.3960303068161011), ('ths', 0.39152267575263977)])
('is', [("isn't", 0.7290782928466797), ('isnt', 0.6854735612869263), ('keeps', 0.6094664335250854), ('was', 0.5938674211502075), ('gets', 0.5444707870483398), ("she's", 0.5371303558349609), ("wasn't", 0.5240029692649841), ("he's", 0.5117640495300293), ('shes', 0.5107032060623169), ('hes', 0.5040029287338257)])
('a', [('another', 0.6717653274536133), ('this', 0.40285879373550415), ('the', 0.3975605070590973), ('every', 0.3918810486793518), ('parking', 0.37662768363952637), ('huge', 0.37332087755203247), ('that', 0.349668025970459), ('massive', 0.33672189712524414), ("yesterday's", 0.3366018533706665), ('something', 0.33431485295295715)])
('test', [('exam', 0.8069786429405212), 

In [172]:
import itertools
import string
z = [list(zip(itertools.repeat(x[0]),[y[0] for y in x[1] if not any((y[0] == p for p in string.punctuation))])) for x in words]
z = list(itertools.chain(*z))
z = [list(x) for x in z]

In [173]:
print(z)

[['this', 'that'], ['this', 'which'], ['this', 'the'], ['this', 'it'], ['this', 'every'], ['this', 'next'], ['this', 'another'], ['this', 'a'], ['this', 'today'], ['this', 'ths'], ['is', "isn't"], ['is', 'isnt'], ['is', 'keeps'], ['is', 'was'], ['is', 'gets'], ['is', "she's"], ['is', "wasn't"], ['is', "he's"], ['is', 'shes'], ['is', 'hes'], ['a', 'another'], ['a', 'this'], ['a', 'the'], ['a', 'every'], ['a', 'parking'], ['a', 'huge'], ['a', 'that'], ['a', 'massive'], ['a', "yesterday's"], ['a', 'something'], ['test', 'exam'], ['test', 'maths'], ['test', 'assignment'], ['test', 'math'], ['test', 'essay'], ['test', 'chem'], ['test', 'lesson'], ['test', 'paper'], ['test', 'physics'], ['test', 'biology'], ['of', 'current'], ['of', "world's"], ['of', 'valuable'], ['of', 'stanley'], ['of', 'involving'], ['of', 'recent'], ['of', 'gun'], ['of', 'led'], ['of', 'ov'], ['of', 'tree'], ['the', 'our'], ['the', 'their'], ['the', 'teh'], ['the', 'this'], ['the', 'security'], ['the', 'my'], ['the', 'i

In [133]:
l = list(set(np.array([x[0] for x in words]).flatten()))

In [134]:
print(l)

['test', 'is', 'a', 'this', 'word', 'of', 'stuff', 'for', 'and', 'negativity', '.', 'the']


In [70]:
random.shuffle(c)
nodes = [{'id':x, 'color':y} for x,y in list(zip(l,c))]
print(nodes)

[{'id': 'cuz', 'color': '#00BFFF'}, {'id': 'because', 'color': '#FF7F50'}, {'id': 'word', 'color': '#FFE4E1'}, {'id': '..', 'color': '#A0522D'}, {'id': 'for', 'color': '#F5DEB3'}, {'id': 'but', 'color': '#DC143C'}, {'id': 'cause', 'color': '#FFFFFF'}, {'id': 'test', 'color': '#228B22'}, {'id': 'is', 'color': '#F5F5DC'}, {'id': 'this', 'color': '#6A5ACD'}, {'id': 'cos', 'color': '#FFEBCD'}, {'id': 'that', 'color': '#FAEBD7'}, {'id': 'of', 'color': '#DDA0DD'}, {'id': ',', 'color': '#8B0000'}, {'id': 'a', 'color': '#FFFAF0'}, {'id': 'stuff', 'color': '#32CD32'}, {'id': 'the', 'color': '#A52A2A'}, {'id': '...', 'color': '#483D8B'}, {'id': 'and', 'color': '#6B8E23'}, {'id': 'negativity', 'color': '#008B8B'}, {'id': '.', 'color': '#DEB887'}]


In [57]:
from matplotlib import colors as mcolors
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)


In [58]:
print(colors)

{'b': (0, 0, 1), 'g': (0, 0.5, 0), 'r': (1, 0, 0), 'c': (0, 0.75, 0.75), 'm': (0.75, 0, 0.75), 'y': (0.75, 0.75, 0), 'k': (0, 0, 0), 'w': (1, 1, 1), 'aliceblue': '#F0F8FF', 'antiquewhite': '#FAEBD7', 'aqua': '#00FFFF', 'aquamarine': '#7FFFD4', 'azure': '#F0FFFF', 'beige': '#F5F5DC', 'bisque': '#FFE4C4', 'black': '#000000', 'blanchedalmond': '#FFEBCD', 'blue': '#0000FF', 'blueviolet': '#8A2BE2', 'brown': '#A52A2A', 'burlywood': '#DEB887', 'cadetblue': '#5F9EA0', 'chartreuse': '#7FFF00', 'chocolate': '#D2691E', 'coral': '#FF7F50', 'cornflowerblue': '#6495ED', 'cornsilk': '#FFF8DC', 'crimson': '#DC143C', 'cyan': '#00FFFF', 'darkblue': '#00008B', 'darkcyan': '#008B8B', 'darkgoldenrod': '#B8860B', 'darkgray': '#A9A9A9', 'darkgreen': '#006400', 'darkgrey': '#A9A9A9', 'darkkhaki': '#BDB76B', 'darkmagenta': '#8B008B', 'darkolivegreen': '#556B2F', 'darkorange': '#FF8C00', 'darkorchid': '#9932CC', 'darkred': '#8B0000', 'darksalmon': '#E9967A', 'darkseagreen': '#8FBC8F', 'darkslateblue': '#483D8B

In [59]:
c = [x for x in colors.values() if isinstance(x,str)]

In [60]:
print(c)

['#F0F8FF', '#FAEBD7', '#00FFFF', '#7FFFD4', '#F0FFFF', '#F5F5DC', '#FFE4C4', '#000000', '#FFEBCD', '#0000FF', '#8A2BE2', '#A52A2A', '#DEB887', '#5F9EA0', '#7FFF00', '#D2691E', '#FF7F50', '#6495ED', '#FFF8DC', '#DC143C', '#00FFFF', '#00008B', '#008B8B', '#B8860B', '#A9A9A9', '#006400', '#A9A9A9', '#BDB76B', '#8B008B', '#556B2F', '#FF8C00', '#9932CC', '#8B0000', '#E9967A', '#8FBC8F', '#483D8B', '#2F4F4F', '#2F4F4F', '#00CED1', '#9400D3', '#FF1493', '#00BFFF', '#696969', '#696969', '#1E90FF', '#B22222', '#FFFAF0', '#228B22', '#FF00FF', '#DCDCDC', '#F8F8FF', '#FFD700', '#DAA520', '#808080', '#008000', '#ADFF2F', '#808080', '#F0FFF0', '#FF69B4', '#CD5C5C', '#4B0082', '#FFFFF0', '#F0E68C', '#E6E6FA', '#FFF0F5', '#7CFC00', '#FFFACD', '#ADD8E6', '#F08080', '#E0FFFF', '#FAFAD2', '#D3D3D3', '#90EE90', '#D3D3D3', '#FFB6C1', '#FFA07A', '#20B2AA', '#87CEFA', '#778899', '#778899', '#B0C4DE', '#FFFFE0', '#00FF00', '#32CD32', '#FAF0E6', '#FF00FF', '#800000', '#66CDAA', '#0000CD', '#BA55D3', '#9370DB'